In [2]:
using LinearAlgebra
using Turing
using LimberJack
using Pkg
Pkg.add("ForwardDiff")

   Resolving package versions...
    Updating `~/cosmo_GP/Project.toml`
⌅ [f6369f11] + ForwardDiff v0.10.38
  No Changes to `~/cosmo_GP/Manifest.toml`


In [3]:
function CC()
    z = [
        0.07, 0.09, 0.12, 0.17, 0.179, 0.199, 0.2,
        0.27, 0.28, 0.352, 0.38, 0.3802, 0.4,
        0.4004, 0.4247, 0.44, 0.4497, 0.47, 0.4783,
        0.48, 0.51, 0.593, 0.6, 0.61, 0.68, 0.73,
        0.781, 0.875, 0.88, 0.9, 1.037, 1.3,
        1.363, 1.43, 1.53, 1.75, 1.965
    ]

    data = [
        69.0, 69.0, 68.6, 83.0, 75.0, 75.0, 72.9,
        77.0, 88.8, 83.0, 81.5, 83.0, 95.0, 77.0,
        87.1, 82.6, 92.8, 89.0, 80.9, 97.0, 90.4,
        104.0, 87.9, 97.3, 92.0, 97.3, 105.0,
        125.0, 90.0, 117.0, 154.0, 168.0, 160.0,
        177.0, 140.0, 202.0, 186.5
    ]

    err = [
        19.6, 12.0, 26.2, 8.0, 4.0, 5.0, 29.6,
        14.0, 36.6, 14.0, 1.9, 13.5, 17.0, 10.2,
        11.2, 7.8, 12.9, 23.0, 9.0, 62.0, 1.9,
        13.0, 6.1, 2.1, 8.0, 7.0, 12.0, 17.0,
        40.0, 23.0, 20.0, 17.0, 33.6, 18.0,
        14.0, 40.0, 50.4
    ]

    cov = zeros(length(z), length(z))
    for i in 1:length(z)
        cov[i, i] = err[i]^2
    end

    return (
        data_name = "CC",
        z = z,
        data = data,
        cov = cov
    )
end


CC (generic function with 1 method)

In [4]:
function BOSSDR12_1()
    z = [0.38, 0.51, 0.61]
    data = [1512.39, 1975.22, 2306.68, 81.2087, 90.9029, 98.9647, 0.49749, 0.457523, 0.436148]
    cov = [3.63049e+00 1.80306e+00 9.19842e-01 9.71342e+00 7.75546e+00 5.97897e+00 2.79185e-02 1.24050e-02 4.75548e-03; 
           1.80306e+00 3.77146e+00 2.21471e+00 4.85105e+00 1.19729e+01 9.73184e+00 9.28354e-03 2.22588e-02 1.05956e-02;
           9.19842e-01 2.21471e+00 4.37982e+00 2.43394e+00 6.71715e+00 1.60709e+01 1.01870e-03 9.71991e-03 2.14133e-02;
           9.71342e+00 4.85105e+00 2.43394e+00 5.00049e+02 2.94536e+02 1.42011e+02 3.91498e-01 1.51597e-01 4.36366e-02;
           7.75546e+00 1.19729e+01 6.71715e+00 2.94536e+02 7.02299e+02 4.32750e+02 1.95890e-01 3.88996e-01 1.81786e-01;
           5.97897e+00 9.73184e+00 1.60709e+01 1.42011e+02 4.32750e+02 1.01718e+03 3.40803e-02 2.46111e-01 4.78570e-01;
           2.79185e-02 9.28354e-03 1.01870e-03 3.91498e-01 1.95890e-01 3.40803e-02 2.03355e-03 8.11829e-04 2.64615e-04;
           1.24050e-02 2.22588e-02 9.71991e-03 1.51597e-01 3.88996e-01 2.46111e-01 8.11829e-04 1.42289e-03 6.62824e-04;
           4.75548e-03 1.05956e-02 2.14133e-02 4.36366e-02 1.81786e-01 4.78570e-01 2.64615e-04 6.62824e-04 1.18576e-03]
    return (data_name = "BOSSDR12", z = z, data = data, cov = cov)
end

BOSSDR12_1 (generic function with 1 method)

In [5]:
function BOSSDR12()
    z = [0.38, 0.51, 0.61]
    data = [81.2087, 90.9029, 98.9647]
    cov = [5.00049e+02 2.94536e+02 1.42011e+02; 2.94536e+02 7.02299e+02 4.32750e+02; 1.42011e+02 4.32750e+02 1.01718e+03]
    return (data_name = "BOSSDR12", z = z, data = data, cov = cov)
end

BOSSDR12 (generic function with 1 method)

In [6]:
cc = CC();
bossdr12 = BOSSDR12();

In [7]:
data  = [bossdr12.data; cc.data];

In [8]:
cov = zeros(length(bossdr12.data)+ length(cc.data), length(bossdr12.data)+ length(cc.data));
for i in 1:length(bossdr12.data)
    for j in 1:length(bossdr12.data)
        cov[i,j] = bossdr12.cov[i,j]
    end
end

for i in 1:length(cc.data)
    for j in 1:length(cc.data)
        cov[i+length(bossdr12.data), j+length(bossdr12.data)] = cc.cov[i,j]
    end
end


In [9]:
using Turing
@model function model(data)
    Ωm ~ Uniform(0.2, 0.6)
    Ωb ~ Uniform(0.028, 0.065)
    h ~ Uniform(0.60, 0.82)
    zs = [bossdr12.z; cc.z]
    cosmo = Cosmology(Ωm= Ωm,  Ωb=Ωb, h=h, ns=0.96, σ8=0.81,
                          tk_mode=:EisHu,
                          pk_mode=:Halofit)

    theory = cosmo.cpar.h*100*Ez(cosmo, zs)
    data ~ MvNormal(theory, cov)
end

model (generic function with 2 methods)

In [10]:
z = [bossdr12.z; cc.z]
cos = Cosmology(Ωm= 0.5,  Ωb=0.04, h=0.7, ns=0.96, σ8=0.81, tk_mode=:EisHu, pk_mode=:Halofit)
th = cos.cpar.h*100*Ez(cos, z)
println(size(z))
println(size(data))
println(size(th))
println(size(cov))

(40,)
(40,)
(40,)
(40, 40)


In [11]:
# Condition model on data
cond_model = model(data)

# # Define sampler
# # nadapts = 500
# TAP = 0.65
# sampler = NUTS(TAP)

# # Sample model 
# iterations = 10
# chain = sample(cond_model, sampler, iterations) 

DynamicPPL.Model{typeof(model), (:data,), (), (), Tuple{Vector{Float64}}, Tuple{}, DynamicPPL.DefaultContext}(Main.model, (data = [81.2087, 90.9029, 98.9647, 69.0, 69.0, 68.6, 83.0, 75.0, 75.0, 72.9  …  125.0, 90.0, 117.0, 154.0, 168.0, 160.0, 177.0, 140.0, 202.0, 186.5],), NamedTuple(), DynamicPPL.DefaultContext())

In [ ]:
using Turing
using ForwardDiff
# Sample model 
iterations = 10
chain = sample(cond_model, NUTS(1, 0.65, init_ϵ=0.1, max_depth=8), iterations) 

In [ ]:
# Define sampler
nadapts = 500
TAP = 0.65
sampler2 = NUTS(nadapts, TAP)

# Sample model 
iterations2 = 1000
chain2 = sample(cond_model, sampler2, iterations2) 
     

Sampling   0%|                                          |  ETA: N/A
┌ Info: Found initial step size
│   ϵ = 0.025
└ @ Turing.Inference /home/vis09/.julia/packages/Turing/wjl7M/src/inference/hmc.jl:192
Sampling   0%|▎                                         |  ETA: 6:41:31